<a href="https://colab.research.google.com/github/ObjectMatrix/google-colab-notebook/blob/main/vicuna_qLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
!pip install -q -U accelerate
!pip install -q -U datasets!
# !pip install llama_cpp
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# from llama_cpp import Llama
# import random

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
# model_name = "/content/drive/MyDrive/models/vicuna-7b-1.1.ggmlv3.q4_0.bin"
model_name = "EleutherAI/gpt-neox-20b"

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


- load_in_4bit: The model will be loaded in the memory with 4-bit precision.

- bnb_4bit_use_double_quant: We will do the double quantization proposed by QLoRa.

- bnb_4bit_quant_type: This is the type of quantization. “nf4” stands for 4-bit NormalFloat.

- bnb_4bit_compute_dtype: While we load and store the model in 4-bit, we will partially dequantize it when needed and do all the computations with a 16-bit precision (bfloat16).

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

So now we can load the model in 4-bit:

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map={"":0})

Then, we enable gradient checkpointing:

In [ ]:
model.gradient_checkpointing_enable()

Preprocessing the GPT model for LoRa
This is where we use PEFT. We prepare the model for LoRa, adding trainable adapters for each layer.

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In LoraConfig, you can play with r, alpha, and dropout to obtain better results on your task. You can find more options and details in the PEFT repository.

With LoRa, we add only 8 million parameters. We will only train these parameters and freeze everything else. Fine-tuning should be fast.

Get your dataset ready
For this demo, I use the “english_quotes” dataset. This is a dataset made of famous quotes distributed under a CC BY 4.0 license.

https://towardsdatascience.com/qlora-fine-tune-a-large-language-model-on-your-gpu-27bed5a03e2b

In [ ]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Fine-tuning GPT-NeoX-20B with QLoRa
Finally, the fine-tuning with Hugging Face Transformers is very standard.



In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=2,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train()

GPT Inference with QLoRa
The QLoRa model we fine-tuned can be directly used with the standard Hugging Face Transformers’ inference, as follows:

In [ ]:
text = "Ask not what your country"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))